<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-19 11:43:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-19 11:43:41 (61.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-19 11:43:44--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.139.128, 74.125.141.128, 173.194.210.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.139.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-09-19 11:43:44 (133 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df),len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:1 (real disaster)
Text
 @SourMashNumber7 @tomfromireland @rfcgeom66 @BBCTalkback They didn't succeed the other two times either. Bomb didn't detonate&amp;Shots missed. 

---

target:0 (not a disaster)
Text
 China's Stock Market Crash: Are There Gems In The Rubble?: ChinaÛªs stock market crash this summer has sparked ... http://t.co/2OqSGZqlbz 

---

target:1 (real disaster)
Text
 going to redo my nails and watch behind the scenes of desolation of smaug ayyy 

---

target:0 (not a disaster)
Text
 My room looks like a tornado passed through it and my OCD is not having it. 

---

target:1 (real disaster)
Text
 My back is so sunburned :( 

---



## splitting the data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

In [15]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [16]:
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 someone just reblogged a picture of self harm scars oh please its 2015 can we stop       
 Vectorized version:
 [[ 475   29 9301    3 1380    6  762  396 3441  392  170   37  208   71
    46]]


In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [20]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @Zojadelin you literally almost had a head on collision with us today on pilot knob         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.04963759,  0.00695189, -0.0473756 , ...,  0.04273747,
          0.02126391, -0.00397717],
        [-0.01597722, -0.02670087,  0.00409678, ...,  0.03356806,
         -0.00923656, -0.02864054],
        [-0.02236483,  0.03706293,  0.00299126, ...,  0.04516621,
          0.02864509, -0.03563219],
        ...,
        [ 0.02330071, -0.04023955,  0.01195326, ...,  0.04618986,
          0.02612264, -0.01756601],
        [ 0.02982174,  0.00673227,  0.02201923, ..., -0.00834076,
         -0.02606033, -0.04774963],
        [ 0.04104184, -0.02280544, -0.03062913, ..., -0.01513471,
         -0.00851408, -0.04295882]]], dtype=float32)>

In [22]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

('@Zojadelin you literally almost had a head on collision with us today on pilot knob',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.04963759,  0.00695189, -0.0473756 , -0.00808641,  0.01944996,
         0.02207186,  0.00282284,  0.04765023, -0.04574596, -0.02871702,
        -0.0282367 ,  0.02512277,  0.02456922,  0.02471378,  0.00301783,
        -0.006576  ,  0.03892738, -0.02412012,  0.00575498,  0.01225729,
         0.01401186, -0.00922875,  0.04548566, -0.0108803 , -0.04810833,
         0.00535885,  0.00960701,  0.03913281,  0.03393719,  0.01029456,
         0.0093488 , -0.04991739, -0.01257131, -0.03636987, -0.02748543,
         0.04540339,  0.03693639, -0.04787586, -0.01262792, -0.03660866,
        -0.01135117, -0.04449179, -0.01040376, -0.01972042,  0.01967864,
        -0.02767302,  0.02486123,  0.02849176, -0.02893412, -0.00485425,
         0.0313797 ,  0.00622495,  0.04455233, -0.00268924, -0.01662014,
         0.00689958, -0.03852063,  0.00864434, -0.04982516, 

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [25]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [27]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

In [29]:
SAVE_DIR = 'model_logs'

In [30]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [31]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [32]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220919-114346
Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6111 - accuracy: 0.6944 - val_loss: 0.5366 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4409 - accuracy: 0.8222 - val_loss: 0.4752 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.3475 - accuracy: 0.8613 - val_loss: 0.4568 - val_accuracy: 0.7953
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.2847 - accuracy: 0.8885 - val_loss: 0.4653 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 5s 25ms/step - loss: 0.2381 - accuracy: 0.9140 - val_loss: 0.4805 - val_accuracy: 0.7913


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 0.4805 - accuracy: 0.7913


[0.48049691319465637, 0.7913385629653931]

In [34]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.31478837],
       [0.8107867 ],
       [0.99745065],
       [0.12406948],
       [0.12139517],
       [0.9368982 ],
       [0.9047222 ],
       [0.99207854],
       [0.96062124],
       [0.26374823]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [36]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7913385826771654,
 'precision': 0.7986313296680336,
 'recall': 0.7913385826771654,
 'f1': 0.7877263030663991}

In [37]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([False, False, False,  True])

## Visualising learned embeddings

In [38]:
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [41]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [42]:
# Let's download the saved embeddings locally
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

## Recurrent Neural Networks

###Model 2 : LSTM

In [43]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
# x = layers.LSTM(64, activation="tanh", return_sequences=True)(x) # use return_sequences=True if you want to stack recurrent layers 
# print(f"After LSTM cell with return_sequences=True: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(f"After LSTM cell: {x.shape}")
# x = layers.Dense(64, activation="relu")(x) # optional dense layer to have on top of LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 64)


In [44]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [45]:
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs= 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                 experiment_name = "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220919-114417
Epoch 1/5
215/215 [==============================] - 15s 49ms/step - loss: 0.2274 - accuracy: 0.9153 - val_loss: 0.5677 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 9s 43ms/step - loss: 0.1574 - accuracy: 0.9412 - val_loss: 0.5672 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 9s 44ms/step - loss: 0.1316 - accuracy: 0.9534 - val_loss: 0.6807 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 11s 50ms/step - loss: 0.1044 - accuracy: 0.9602 - val_loss: 0.8320 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 12s 57ms/step - loss: 0.0882 - accuracy: 0.9670 - val_loss: 0.8248 - val_accuracy: 0.7690


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

((762, 1), array([[0.01786107],
        [0.8223864 ],
        [0.9997641 ],
        [0.12072086],
        [0.00180042],
        [0.99943197],
        [0.8968291 ],
        [0.99982935],
        [0.99976027],
        [0.4075573 ]], dtype=float32))

In [47]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_2_results = calculate_result(y_true= val_labels,
                                   y_pred = model_2_preds)
model_2_results

{'accuracy': 0.7690288713910761,
 'precision': 0.7693974434525301,
 'recall': 0.7690288713910761,
 'f1': 0.7676527036976099}

In [49]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

In [50]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation= 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs, name='model_3_GRU')

In [51]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [52]:
history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs =5,
                        validation_data =(val_sentences, val_labels),
                        callbacks= [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220919-114516
Epoch 1/5
215/215 [==============================] - 11s 33ms/step - loss: 0.1644 - accuracy: 0.9342 - val_loss: 0.7308 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0835 - accuracy: 0.9701 - val_loss: 0.9261 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 6s 29ms/step - loss: 0.0679 - accuracy: 0.9742 - val_loss: 0.8877 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 8s 39ms/step - loss: 0.0545 - accuracy: 0.9785 - val_loss: 1.4220 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 11s 51ms/step - loss: 0.0552 - accuracy: 0.9756 - val_loss: 1.2709 - val_accuracy: 0.7756


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[1.9205847e-01],
       [8.1738019e-01],
       [9.9980521e-01],
       [5.4228425e-02],
       [1.1055987e-05],
       [9.9944675e-01],
       [9.4113481e-01],
       [9.9992114e-01],
       [9.9987042e-01],
       [7.2285092e-01]], dtype=float32)

In [54]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
model_3_results = calculate_result(y_true= val_labels,
                                   y_pred = model_3_preds)
model_3_results

{'accuracy': 0.7755905511811023,
 'precision': 0.7753516653505942,
 'recall': 0.7755905511811023,
 'f1': 0.7748751194086603}

### Model 4: Bidirectional RNN

In [56]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences= True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs,outputs,name='model_4_bidirectional')

In [57]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [58]:
model_4.compile(loss= 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [59]:
history_4 = model_4.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data = (val_sentences,val_labels),
                        callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                 experiment_name= 'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20220919-114601
Epoch 1/5
215/215 [==============================] - 18s 52ms/step - loss: 0.1110 - accuracy: 0.9653 - val_loss: 1.0050 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0557 - accuracy: 0.9755 - val_loss: 1.0091 - val_accuracy: 0.7559
Epoch 3/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0459 - accuracy: 0.9784 - val_loss: 1.3988 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 8s 37ms/step - loss: 0.0486 - accuracy: 0.9800 - val_loss: 1.2346 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 8s 39ms/step - loss: 0.0412 - accuracy: 0.9806 - val_loss: 1.4453 - val_accuracy: 0.7703


In [60]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[3.5497546e-04],
       [8.0367780e-01],
       [9.9998361e-01],
       [2.6607370e-01],
       [6.6923822e-05],
       [9.9935722e-01],
       [8.2190597e-01],
       [9.9999231e-01],
       [9.9998665e-01],
       [9.9665546e-01]], dtype=float32)

In [61]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
model_4_results = calculate_result(y_true= val_labels,
                                   y_pred = model_4_preds)
model_4_results

{'accuracy': 0.7703412073490814,
 'precision': 0.7729377683268125,
 'recall': 0.7703412073490814,
 'f1': 0.7677842762403819}

### Model 5:conv1D

In [63]:
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(['this is a test sentence']))
conv_1d = layers.Conv1D(filters = 32,
                        kernel_size = 5,
                        activation = 'relu',
                        padding = 'valid')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

In [64]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters= 64, kernel_size=5,strides=1,activation= 'relu',padding= 'valid')(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation= 'sigmoid')(x)
model_5 = tf.keras.Model(inputs, outputs, name= 'model_5_Conv1D')

model_5.compile(loss= 'binary_crossentropy',
                optimizer= tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [65]:
history_5 = model_5.fit(train_sentences,
                        train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                 'Conv1D')])

Saving TensorBoard log files to: model_logs/Conv1D/20220919-114652
Epoch 1/5
215/215 [==============================] - 5s 21ms/step - loss: 0.1202 - accuracy: 0.9609 - val_loss: 0.8936 - val_accuracy: 0.7703
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0754 - accuracy: 0.9717 - val_loss: 1.0662 - val_accuracy: 0.7612
Epoch 3/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0596 - accuracy: 0.9771 - val_loss: 1.1393 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0548 - accuracy: 0.9781 - val_loss: 1.2191 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0525 - accuracy: 0.9785 - val_loss: 1.2491 - val_accuracy: 0.7572


In [66]:
model_5_pred_probs=model_5.predict(val_sentences)
model_5_pred_probs[:10]

array([[3.32579017e-02],
       [9.31826472e-01],
       [9.99942780e-01],
       [3.65534723e-02],
       [1.10054174e-07],
       [9.99063730e-01],
       [9.92854476e-01],
       [9.99914706e-01],
       [9.99998987e-01],
       [8.12469006e-01]], dtype=float32)

In [67]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [68]:
model_5_results= calculate_result(y_true= val_labels,
                                  y_pred = model_5_preds)
model_5_results

{'accuracy': 0.7572178477690289,
 'precision': 0.7585989210360964,
 'recall': 0.7572178477690289,
 'f1': 0.7549472494674683}

## Model 6: Tensorflow Hub pretrained sentence encoder

In [70]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed([sample_sentence,
                      "when you call the universal sentence encoder on sentence ,it turns it  into numbers "])
print(embed_sample[0][:50])

tf.Tensor(
[-0.00767769  0.03333809  0.0320662  -0.02125602  0.04300849  0.08399193
  0.02452653  0.05380183 -0.02588227 -0.00499588  0.01928363 -0.00056099
  0.01276441  0.09088546  0.0562213  -0.04887728  0.05191502 -0.04136144
 -0.01161554 -0.05452844 -0.01740017  0.0045459   0.00692502 -0.01381139
 -0.00961125 -0.03933032  0.0197613  -0.01115342 -0.03671153  0.01013187
 -0.04107149  0.03479797 -0.03814119 -0.00185953  0.01663749 -0.08907217
  0.00592126  0.04859087 -0.03741663 -0.08817419  0.00334023  0.00904682
 -0.01701778  0.06431309 -0.10647684 -0.0381299  -0.02107327 -0.02888192
 -0.03679271  0.01302323], shape=(50,), dtype=float32)


In [80]:
sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                                        input_shape=[],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = "USE")

In [81]:
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation= 'relu'),
    layers.Dense(1, activation = 'sigmoid',name= 'output_layer')
],name = 'model_6_USE')

model_6.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [82]:
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_10 (Dense)            (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [83]:
history_6 = model_6.fit(train_sentences,
                        train_labels,
                        epochs = 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks =[create_tensorboard_callback(SAVE_DIR,
                                                                'tf_hub_sentence_encoder')])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20220919-122357
Epoch 1/5
215/215 [==============================] - 9s 23ms/step - loss: 0.5029 - accuracy: 0.7867 - val_loss: 0.4513 - val_accuracy: 0.7940
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4157 - accuracy: 0.8135 - val_loss: 0.4442 - val_accuracy: 0.8084
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.4004 - accuracy: 0.8218 - val_loss: 0.4343 - val_accuracy: 0.8084
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.3926 - accuracy: 0.8237 - val_loss: 0.4301 - val_accuracy: 0.8150
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.3864 - accuracy: 0.8288 - val_loss: 0.4338 - val_accuracy: 0.8097


In [84]:
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

array([[0.15056372],
       [0.7298138 ],
       [0.98550975],
       [0.16617277],
       [0.65866107],
       [0.69605386],
       [0.98020065],
       [0.9744202 ],
       [0.94169796],
       [0.08057344]], dtype=float32)

In [86]:
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [89]:
model_6_results = calculate_result(y_true= val_labels,
                                y_pred = model_6_preds)
model_6_results

{'accuracy': 0.8097112860892388,
 'precision': 0.8136104189749119,
 'recall': 0.8097112860892388,
 'f1': 0.8075926860277451}

In [90]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 7: TF hub pretrained USE but with 10 % of the data

In [93]:
train_10_percent = train_df_shuffled[['text','target']].sample(frac=0.1, random_state = 42)
train_sentences_10_percent = train_10_percent['text'].to_list()
train_labels_10_percent = train_10_percent['target'].to_list()

len(train_sentences_10_percent), len(train_labels_10_percent)

(761, 761)

In [95]:
train_10_percent['target'].value_counts()

0    413
1    348
Name: target, dtype: int64

In [97]:
# Let's build a model the same as model_6
# model_7 = tf.keras.models.clone_model(model_6)
model_7 = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid", name="output_layer")                               
], name="model_7_USE")

# Compile model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary (will be same as model_6)
model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_11 (Dense)            (None, 64)                32832     
                                                                 
 output_layer (Dense)        (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [98]:
history_7 = model_7.fit(train_sentences_10_percent,
                        train_labels_10_percent,
                        epochs = 5, 
                        validation_data = (val_sentences , val_labels),
                        callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                 'tf_hub_sentence_encoder_10_percent')])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent/20220919-130244
Epoch 1/5
24/24 [==============================] - 3s 46ms/step - loss: 0.6746 - accuracy: 0.6518 - val_loss: 0.6382 - val_accuracy: 0.8084
Epoch 2/5
24/24 [==============================] - 1s 25ms/step - loss: 0.6064 - accuracy: 0.7924 - val_loss: 0.5621 - val_accuracy: 0.7992
Epoch 3/5
24/24 [==============================] - 1s 26ms/step - loss: 0.5304 - accuracy: 0.7976 - val_loss: 0.4909 - val_accuracy: 0.8071
Epoch 4/5
24/24 [==============================] - 1s 26ms/step - loss: 0.4716 - accuracy: 0.8029 - val_loss: 0.4429 - val_accuracy: 0.8176
Epoch 5/5
24/24 [==============================] - 1s 26ms/step - loss: 0.4312 - accuracy: 0.8147 - val_loss: 0.4106 - val_accuracy: 0.8294


In [99]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

array([[0.24001682],
       [0.8100462 ],
       [0.9133613 ],
       [0.31515145],
       [0.7975489 ],
       [0.81227624],
       [0.8965697 ],
       [0.93685377],
       [0.82232237],
       [0.07328638]], dtype=float32)

In [100]:
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [104]:
model_7_results = calculate_result(y_true = val_labels,
                                    y_pred = model_7_preds)
model_7_results

{'accuracy': 0.8293963254593176,
 'precision': 0.8292333273848194,
 'recall': 0.8293963254593176,
 'f1': 0.8291707801295185}

In [105]:
model_6_results

{'accuracy': 0.8097112860892388,
 'precision': 0.8136104189749119,
 'recall': 0.8097112860892388,
 'f1': 0.8075926860277451}